<a href="https://colab.research.google.com/github/Gabor132/QuarantineSportsClassifier/blob/master/src/QuarantineSportsCreateModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quarantine Sports Classifier 

## Imports

In [0]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical


## Constants

In [0]:
learning_rate = 0.01
testing_data_percentage = 0.2
nr_of_frames_per_sequence = 15
nr_of_keypoints = 25
nr_of_values_per_keypoint = 3

all_files_names = ["video1.json", "video2.json", "video3.json"]

# Dataset paths
DRAGOS_COLAB_DATASET_PATH = "/content/drive/My Drive/QuarantineSportsDatasets/Dataset/OpenPoseDataset/"
LOCAL_DATASET_PATH = "../datasets/"

# Model paths
DRAGOS_COLAB_MODEL_PATH = "/content/drive/My Drive/QuarantineSportsDatasets/Model/"
LOCAL_MODEL_PATH = "../"


In [0]:
current_dataset_path = DRAGOS_COLAB_DATASET_PATH
current_model_path = DRAGOS_COLAB_MODEL_PATH

In [0]:
dataset_paths = []
for path in all_files_names:
  dataset_path = current_dataset_path + path
  dataset_paths.append(dataset_path)

print("All files found: {}".format(dataset_paths))

## Load Datasets and create X and Y

## Important to know

1.   Out of open pose we extract for all frames all the 25 keypoints (each keypoint contains the X, Y and score)
2.   The Category exists per frame and is 0/1 as in Wrong/Correct as in Not a Push-Up Frame/Is a Push Up Frame


In [0]:
y_total = None
x_total = None
for (index, path) in enumerate(dataset_paths):
  print("For file at {}".format(path))
  df = pd.read_json(path)
  keypoints = df['Keypoints'].values
  file_y = df['Category'].values
  file_x = []
  for k in keypoints:
      if k != None:
          newK = np.reshape(np.asarray(k), (25,3))
          file_x.append(newK)
      else:
          file_x.append(np.reshape(np.zeros(75), (25,3)))
  file_x = np.array(file_x)
  print("X: {} and Y: {}".format(file_x.shape, file_y.shape))
  if np.all(x_total == None):
    x_total = file_x
  else:
    x_total = np.vstack((x_total, file_x))
  if np.all(y_total == None):
    y_total = file_y
  else:
    y_total = np.hstack((y_total, file_y))
print("Total X: {}".format(x_total.shape))
print("Total Y: {}".format(y_total.shape))

## Splitting the dataset into categories (wrong and correct)

In [0]:
# Get Indexes
y_wrong_index = np.where(y_total == 0)
y_correct_index = np.where(y_total == 1)
# Get Values
x_wrong = x_total[y_wrong_index]
y_wrong = y_total[y_wrong_index]
x_correct = x_total[y_correct_index]
y_correct = y_total[y_correct_index]


## Further splitting the frames into sequences

In [0]:
print("Initial shapes")
print("y_wrong: {}".format(y_wrong.shape))
print("y_correct: {}".format(y_correct.shape))
print("x_wrong: {}".format(x_wrong.shape))
print("x_correct: {}".format(x_correct.shape))

class FrameSet:

  def __init__(self, x, y):
    self.x = x
    self.y = y

def get_maximum_number_of_usable_frames(a):
  divider = nr_of_frames_per_sequence
  i = 1
  while (divider * i) < len(a):
    i = i + 1
  return divider * (i-1)

def transform_y_into_frame_sets(a):
  number_of_sets = get_maximum_number_of_usable_frames(a)
  print("Number of sets: {}".format(number_of_sets))
  new_y = None
  if np.any(a == 0):
    new_y = np.zeros((int) (number_of_sets / nr_of_frames_per_sequence))
  else:
    new_y = np.ones((int) (number_of_sets / nr_of_frames_per_sequence))
  return new_y

def transform_x_into_frame_sets(a):
  number_of_sets = get_maximum_number_of_usable_frames(a)
  print("Number of sets: {}".format(number_of_sets))
  sub_group_size = a[0:number_of_sets]
  return np.reshape(sub_group_size, ((int) (number_of_sets / nr_of_frames_per_sequence), nr_of_frames_per_sequence, nr_of_keypoints * nr_of_values_per_keypoint))

y_wrong = transform_y_into_frame_sets(y_wrong)
y_correct = transform_y_into_frame_sets(y_correct)
x_wrong = transform_x_into_frame_sets(x_wrong)
x_correct = transform_x_into_frame_sets(x_correct)
print("After reshaping")
print("y_wrong: {}".format(y_wrong.shape))
print("y_correct: {}".format(y_correct.shape))
print("x_wrong: {}".format(x_wrong.shape))
print("x_correct: {}".format(x_correct.shape))

## Recompine all wrong and correct arrays

In [0]:
# Only stack them when they are not empty
x_processed = None
y_processed = None
x_processed = np.concatenate((x_correct, x_wrong))
y_processed = np.concatenate((y_correct, y_wrong))
y_processed = to_categorical(y_processed)
print(x_processed.shape)
print(y_processed.shape)

## Finally, split them into training and testing datasets

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_processed, y_processed, test_size=0.2)
print("Number of training Frame Sets {} and number of testing Frame sets {}".format(len(x_train), len(x_test)))

## Defining the Model

In [0]:
print(x_train.shape)
model = keras.Sequential()
model.add(keras.layers.LSTM(15, input_shape=(15, 75), activation='softmax'))
model.add(keras.layers.Dropout(0.2, input_shape=(15,)))
#model.add(keras.layers.Conv1D(3, 3, activation='softmax'))
model.add(keras.layers.Dense(2, activation='softmax'))
model.build(input_shape=(1, 75))
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),loss='categorical_crossentropy',
              metrics=[keras.metrics.categorical_crossentropy])
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Saving Model to JSON

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open(current_model_path + "model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(current_model_path + "model.h5")
print("Saved model to disk")

## Finish